In [31]:
from pathlib import Path
import pandas as pd

In [32]:
file_path = Path("archive/movies.csv")
df_movie = pd.read_csv(file_path)

In [33]:
df_movie["released"] = df_movie["released"].str.split("\s+\(").str[0]
#df_movie["released_country"] = df_movie["released"].str.split("\s+\(").str[1]
df_movie["released"] = pd.to_datetime(df_movie['released'])


In [34]:
df_movie ["released"] = df_movie["released"].dt.month_name()

In [35]:
df_movie.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,June,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,July,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,June,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,July,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,July,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [36]:
df_movie = df_movie.drop(columns=['score', 'votes'])

In [37]:
df_movie.head()

,name,rating,genre,year,released,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [80]:
df_movie['trixbudget']= df_movie['gross'] - 2*df_movie['budget']

In [81]:
df_movie = df_movie.dropna()

In [82]:
df_movie.head()

,name,rating,genre,year,released,director,writer,star,country,budget,gross,company,runtime,trixbudget,success
0,The Shining,R,Drama,1980,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,8998772.0,0
1,The Blue Lagoon,R,Adventure,1980,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,49853106.0,1
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,502375067.0,1
3,Airplane!,PG,Comedy,1980,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,76453539.0,1
4,Caddyshack,R,Comedy,1980,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,27846344.0,1


In [83]:
df_movie['success'] = df_movie['trixbudget'].map(lambda x: x>0).astype(int)

In [92]:
df_movie.sort_values(by="gross", ascending=True)

,name,rating,genre,year,released,director,writer,star,country,budget,gross,company,runtime,trixbudget,success
3203,Trojan War,PG-13,Comedy,1997,October,George Huang,Andy Burg,Will Friedle,United States,15000000.0,3.090000e+02,Daybreak,85.0,-2.999969e+07,0
272,Parasite,R,Horror,1982,March,Charles Band,Alan J. Adler,Robert Glaudini,United States,800000.0,2.270000e+03,Embassy Pictures,85.0,-1.597730e+06,0
3681,Ginger Snaps,Not Rated,Drama,2000,May,John Fawcett,Karen Walton,Emily Perkins,Canada,5000000.0,2.554000e+03,Copperheart Entertainment,108.0,-9.997446e+06,0
2434,Philadelphia Experiment II,PG-13,Action,1993,June,Stephen Cornwell,Wallace C. Bennett,Brad Johnson,United States,5000000.0,2.970000e+03,Trimark Pictures,97.0,-9.997030e+06,0
5640,Tanner Hall,R,Drama,2009,January,Francesca Gregorini,Tatiana von Fürstenberg,Rooney Mara,United States,3000000.0,5.073000e+03,Two Prong Lesson,96.0,-5.994927e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7244,Avengers: Infinity War,PG-13,Action,2018,April,Anthony Russo,Christopher Markus,Robert Downey Jr.,United States,321000000.0,2.048360e+09,Marvel Studios,149.0,1.406360e+09,1
6663,Star Wars: Episode VII - The Force Awakens,PG-13,Action,2015,December,J.J. Abrams,Lawrence Kasdan,Daisy Ridley,United States,245000000.0,2.069522e+09,Lucasfilm,138.0,1.579522e+09,1
3045,Titanic,PG-13,Drama,1997,December,James Cameron,James Cameron,Leonardo DiCaprio,United States,200000000.0,2.201647e+09,Twentieth Century Fox,194.0,1.801647e+09,1
7445,Avengers: Endgame,PG-13,Action,2019,April,Anthony Russo,Christopher Markus,Robert Downey Jr.,United States,356000000.0,2.797501e+09,Marvel Studios,181.0,2.085501e+09,1


In [85]:
df_movie_success = df_movie.loc[df_movie['success'] == 1]

df_movie_failed = df_movie.loc[df_movie['success'] == 0]

In [90]:
df_movie_success.tail()

,name,rating,genre,year,released,director,writer,star,country,budget,gross,company,runtime,trixbudget,success
7645,Birds of Prey,R,Action,2020,February,Cathy Yan,Christina Hodson,Margot Robbie,United States,84500000.0,201858461.0,Clubhouse Pictures (II),109.0,32858461.0,1
7646,The Invisible Man,R,Drama,2020,February,Leigh Whannell,Leigh Whannell,Elisabeth Moss,Canada,7000000.0,143151000.0,Universal Pictures,124.0,129151000.0,1
7648,Bad Boys for Life,R,Action,2020,January,Adil El Arbi,Peter Craig,Will Smith,United States,90000000.0,426505244.0,Columbia Pictures,124.0,246505244.0,1
7649,Sonic the Hedgehog,PG,Action,2020,February,Jeff Fowler,Pat Casey,Ben Schwartz,United States,85000000.0,319715683.0,Paramount Pictures,99.0,149715683.0,1
7652,The Eight Hundred,Not Rated,Action,2020,August,Hu Guan,Hu Guan,Zhi-zhong Huang,China,80000000.0,461421559.0,Beijing Diqi Yinxiang Entertainment,149.0,301421559.0,1


In [105]:
df_movie.nunique()

name          5336
rating          10
genre           15
year            41
released        12
director      2063
writer        3205
star          1845
country         50
budget         398
gross         5418
company       1475
runtime        127
trixbudget    5421
success          2
dtype: int64

In [88]:
df_movie_failed.nunique()

name          2829
rating           9
genre           15
year            41
released        12
director      1522
writer        2129
star          1257
country         38
budget         281
gross         2853
company       1022
runtime        113
trixbudget    2854
success          1
dtype: int64

In [119]:
df_movie.groupby(by="country").count().sort_values(by="gross", ascending=False)["gross"].head(20)

country
United States     4319
United Kingdom     492
Canada             111
France             105
Germany             82
Australia           42
Japan               28
China               26
Hong Kong           24
Ireland             21
Italy               19
New Zealand         18
Spain               18
South Korea         13
Mexico              11
Denmark             10
India                8
Czech Republic       7
Sweden               5
Belgium              4
Name: gross, dtype: int64

In [106]:
df_movie.groupby(by="writer").count().sort_values(by="gross", ascending=False)["gross"].head(2000)

writer
Woody Allen              32
Stephen King             29
John Hughes              20
Luc Besson               14
Joel Coen                13
                         ..
Christopher Hawthorne     1
Christopher Isherwood     1
Christopher Kyle          1
Christopher L. Yost       1
Christopher Murphey       1
Name: gross, Length: 2000, dtype: int64

In [70]:
df_movie.groupby(by="star").count().sort_values(by="gross", ascending=False)["gross"].head(100)

star
Nicolas Cage            40
Tom Hanks               38
Robert De Niro          38
Bruce Willis            34
Tom Cruise              33
                        ..
Cate Blanchett          11
Patrick Swayze          11
Burt Reynolds           11
Michael Caine           11
Joseph Gordon-Levitt    11
Name: gross, Length: 100, dtype: int64

In [79]:
df_movie_failed.groupby(by="company").count().sort_values(by="gross", ascending=False)["gross"].head(30)

company
Warner Bros.                               187
Universal Pictures                         178
Columbia Pictures                          177
Paramount Pictures                         150
Twentieth Century Fox                       99
New Line Cinema                             78
Metro-Goldwyn-Mayer (MGM)                   67
Touchstone Pictures                         59
TriStar Pictures                            52
Walt Disney Pictures                        47
Miramax                                     46
Dreamworks Pictures                         41
Castle Rock Entertainment                   36
Orion Pictures                              33
Lionsgate                                   30
Summit Entertainment                        29
Fox 2000 Pictures                           28
Dimension Films                             27
Morgan Creek Entertainment                  27
Screen Gems                                 25
New Regency Productions                     25
Focus